# Test Develop Mock Circuit

In [1]:
import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.mock.cell import Cell

In [2]:
from neuro_dmt.models.bluebrain.circuit.geometry import Position
p = Position(X = 1., Y = 1., Z = 1.)
print(p.as_tuple)
print(p.value)

random_positions =\
    Position.sample(
        (np.array((-1., -1., -1.)), np.array((1., 1., 1.))),
        n = 3)
print(random_positions)

(1.0, 1.0, 1.0)
[1. 1. 1.]
[[-0.91557735  0.82629491 -0.59784094]
 [-0.10825935 -0.73896715  0.08956719]
 [ 0.3758295   0.86509449 -0.12758718]]


In [3]:
example_cell =\
    Cell(
        layer=1,
        position=Position(X=1., Y=2., Z=3.),
        mtype="L23_BTC")
example_cell.as_dict

{'region': 'brain',
 'layer': 1,
 'nucleus': 'not-defined',
 'mtype': 'L23_BTC',
 'etype': 'not-defined',
 'morph_class': 'not-defined',
 'synapse_class': 'INH',
 'x': 1.0,
 'y': 2.0,
 'z': 3.0}

In [4]:
from neuro_dmt.models.bluebrain.circuit.mock.test_develop\
    import circuit_composition, circuit_connectivity
circuit_composition.cell_density.head()

mean  std
mtype  layer                  
L1_DAC 1      3094.333333  0.0
       2         0.000000  0.0
       3         0.000000  0.0
       4         0.000000  0.0
       5         0.000000  0.0

In [5]:
from neuro_dmt.models.bluebrain.circuit.mock import CircuitBuilder
test_circuit_builder =\
    CircuitBuilder(
        composition=circuit_composition,
        connectivity=circuit_connectivity)
test_circuit_builder.get_cell_density( mtype="L1_DAC", layer=1 )

,,mean,std
mtype,layer,,
L1_DAC,1,3094.333333,0.0


In [6]:
print(circuit_composition.cell_density.loc["L1_DAC"])
print(test_circuit_builder.get_number_cells(1, "L1_DAC"))

              mean  std
layer                  
1      3094.333333  0.0
2         0.000000  0.0
3         0.000000  0.0
4         0.000000  0.0
5         0.000000  0.0
6         0.000000  0.0
23


In [7]:
circuit_cells = test_circuit_builder.get_cells()

In [8]:
import pandas as pd
circuit_cells_df =\
    pd.DataFrame([
        cell.as_dict for cell in circuit_cells])
print("number of cells {}".format(circuit_cells_df.shape[0]))
print(circuit_cells_df.head())

number of cells 12237
         etype  layer  morph_class   mtype      nucleus region synapse_class  \
0  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
1  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
2  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
3  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
4  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   

            x            y           z  
0   96.201736  1762.647126   64.541717  
1  240.077672  1828.614957  220.628095  
2  102.333199  1722.321061   55.503750  
3  221.816694  1738.919199  198.745262  
4   45.651688  1806.277845  105.267527  


In [9]:
from neuro_dmt.models.bluebrain.circuit.mock.synapse import Synapse
s = Synapse(pre_gid=1, post_gid=2)
s.as_dict

{'post_gid': 2, 'pre_gid': 1}

In [10]:
cell_collection = test_circuit_builder.get_cell_collection()
cell_collection.get(1, "mtype")

'L1_DAC'

In [11]:
cell_collection.get([1,2], properties=["mtype", "etype"])

,mtype,etype
1,L1_DAC,not-defined
2,L1_DAC,not-defined


In [12]:
from neuro_dmt.models.bluebrain.circuit.mock.circuit import MockCircuit
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
mock_conn =\
    mock_circuit.connectome

In [13]:
mock_cells = mock_circuit.cells
mock_cells.get([1, 3000], "mtype")

1       L1_DAC
3000     L4_PC
Name: mtype, dtype: object

In [14]:
mock_conn = mock_circuit.connectome
print(mock_conn.afferent_gids(2))
print(mock_conn.efferent_gids(14))

[    0    17    33 ... 12229 12231 12235]
[    3     6     8 ... 12228 12233 12234]


In [15]:
post_gid = 0
pre_gid = 0
print(mock_conn._get_pair_synapse_count(pre_gid, post_gid))
print(mock_conn.afferent_adjacency[post_gid][0:10,:])
print(mock_conn._get_synapses(pre_gid, post_gid))
print(mock_conn.pair_synapses(pre_gid, post_gid))
#print(mock_conn.pair_synapses(pre_gid, 1))

3
[[ 0  3]
 [ 1  5]
 [39  2]
 [42  5]
 [43  2]
 [50  6]
 [67 10]
 [72  6]
 [75  3]
 [95  6]]
                                pre_gid  post_gid  synapse_index
pre_gid post_gid synapse_index                                  
0       0        0                    0         0              0
                 1                    0         0              1
                 2                    0         0              2
[(0, 0, 0) (0, 0, 1) (0, 0, 2)]


In [16]:
gid = 10
aff_gids = mock_conn.afferent_gids(gid)
print(len(aff_gids))
aff_syns = mock_conn.afferent_synapses(gid)
print(len(aff_syns))

1939
9732


In [17]:
gid = 10
eff_gids = mock_conn.efferent_gids(gid)
print(len(eff_gids))
eff_syns = mock_conn.efferent_synapses(gid)
#eff_syns = mock_conn._get_properties(
 #   mock_conn._get_synapses(pre_gid=gid),
  #  properties=None)
print(len(eff_syns))

1940
9697


In [18]:
mock_conn.empty_synapse_holder()

,,,pre_gid,post_gid,synapse_index
pre_gid,post_gid,synapse_index,,,


In [19]:
print(mock_conn.pathway_synapses(np.array([pre_gid]), np.array([post_gid])))
print(mock_conn.pathway_synapses(np.array([0, 5, 8]), np.array([0])))
print(mock_conn.pathway_synapses(post_gids=np.array([0])))

[(0, 0, 0) (0, 0, 1) (0, 0, 2)]
[(0, 0, 0) (0, 0, 1) (0, 0, 2)]
[(0, 0, 0) (0, 0, 1) (0, 0, 2) ... (12233, 0, 4) (12233, 0, 5)
 (12233, 0, 6)]
